# Energy in the Chilled Water System

The chilled water system (CWS) at UIUC is analogous to battery storage because the grid demand on campus feels it like a decrease in load because the chilled water is discharged from the tank before the electric chillers are turned on. To calculate the stored energy value in MWh, we can use:

$Q = mc_p\Delta T$ [J]


In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def Q (m, c, delta_T):
    """
    This function calculates the energy stored in joules. 
    
    Parameters:
    -----------
    m : float
        This is the mass of the heated/cooled substance in [kg]
    c : float
        This is the specific heat capacity of the substance in 
        [J/kg C]
    delta_T : float
        This is the change in temperature of the substance. 
    
    Returns:
    --------
    q : float
        The energy stored in the substance. 
    """
    q = m*c*delta_T
    return q


In [3]:
tank_vol = 6.5e6 # gallons

# convert gallons to cubic meters
tank_vol = tank_vol/264

rho_w = 997 # kg/m^3
cp_w = 4186 # j/kg C
dT = 9.1 # C
mass_w = rho_w*tank_vol # kg
Q_cws = Q(mass_w, cp_w, dT)
print("The energy stored in the full CWS tank is {} joules".format(Q_cws))

The energy stored in the full CWS tank is 935072326893.9393 joules


#### Converting Joules to MWh

We can convert Joules to MWh by the following:

$1 W = 1 J/s$ <br>
$1 Wh = 1 J/s * 3600 s/hour$<br>
$1 MWh = 1x10^6 Wh$

So

$ # MWh = # J / (1e6 * 3600)$

In [4]:
Q_MWh = Q_cws / 3.6e9

print("The MWh equivalent in energy storage is {} MWh (thermal)".format(Q_MWh))

The MWh equivalent in energy storage is 259.7423130260943 MWh (thermal)


#### Conversion Efficiency

Chilled water is a form of thermal energy storage. To get the equivalent energy storage in electric energy, a factor of 1/3 is multiplied as an assumption ($\eta = 33\%$)

In [5]:
print("The battery energy equivalent (BEE) is {} MWh (electric)".format(Q_MWh/3))

The battery energy equivalent (BEE) is 86.5807710086981 MWh (electric)


#### Question

This battery equivalent is only true to a certain extent. I probably have to cap the outflow at a certain level, but how much is the question. 

This [document](https://www.districtenergy.org/HigherLogic/System/DownloadDocumentFile.ashx?DocumentFileKey=d7d7ad58-c30a-7f1d-3d74-ab02a2320e6a&forceDialog=0) give some indication of a solution but the reality is I need data from eDNA (enter Mike Marquissee) about the chilled water use on campus. 

I couldn't get information from Mike before the Pandemic struck. So I'm going to use information from the master plan.

In [6]:
import numpy as np
el_eff_ls = np.array([0.637, 0.631, 0.615, 0.580, 0.653, 0.640, 0.640, 0.645, 0.641, 0.676, 0.653, 0.595, 
          1.2, 1.2, 0.676, 0.64, 0.64, 0.64, 0.565, 0.622, 0.634, 0.565])
st_eff_ls = np.array([18,18,11,11])

mean_eleff = el_eff_ls.mean()
mean_steff = st_eff_ls.mean()

print(f"The mean CHW electric efficiency is {round(mean_eleff,2)} KW/TON the mean CHW steam efficiency is {mean_steff} PPH/TON")

The mean CHW electric efficiency is 0.68 KW/TON the mean CHW steam efficiency is 14.5 PPH/TON


This means that for every chiller to be running at maximum capacity simultaneously would require CAPxEFF.

In [7]:
peak_electric = 51860*0.68
peak_electric

35264.8

In [8]:
import pandas as pd
demand = pd.read_csv('./../../data/annual_load2017.csv')
demand

,Unnamed: 0,time,kw
0,0,0,43345.0
1,1,1,43064.0
2,2,2,43035.0
3,3,3,43366.0
4,4,4,43281.0
...,...,...,...
8755,8755,8755,41299.0
8756,8756,8756,41045.0
8757,8757,8757,39193.0
8758,8758,8758,40234.0


In [9]:
demand['kw'].sum()/0.33/1e6

1352.542306069697